In [0]:
import os
import pandas as pd
import requests

In [0]:
# Read environment/config values
# Get parameters from Databricks job run
EMAIL = dbutils.widgets.get("email")  # your email parameter
GITHUB_TOKEN = dbutils.widgets.get("github_pat")  # your GitHub PAT
RETRAIN_PATH = f"/Workspace/Users/{EMAIL}/CREDITCARD/Retraining_Decision/Retrain.csv"

In [0]:
# GitHub configuration (replace these with your own)
GITHUB_REPO = "sajagmathur/ML-Ops-CreditCard-Databricks"   # e.g., "org/mlops-pipeline"
GITHUB_WORKFLOW_FILE = "CI CD Pipleline.yml"      # name of your workflow YAML file in .github/workflows/
GITHUB_API_URL = f"https://api.github.com/repos/{GITHUB_REPO}/actions/workflows/{GITHUB_WORKFLOW_FILE}/dispatches"
GITHUB_BRANCH = "main"

In [0]:
import pandas as pd
from datetime import datetime

print("🔍 Starting retraining decision check...")

try:
    print(f"📂 Attempting to read retraining decision file from: {RETRAIN_PATH}")
    df = pd.read_csv(RETRAIN_PATH)
    print(f"✅ Successfully loaded retraining decision file. Rows found: {len(df)}")
except FileNotFoundError:
    print("❌ Retrain.csv not found. Skipping retraining check and exiting notebook.")
    dbutils.notebook.exit("No retraining")

decision = df.loc[0, 'Retraining_Decision'].strip().upper()
print(f"📝 Retrieved decision from file: '{decision}'")

if decision == "YES":
    print("🔁 Retraining required. Preparing to trigger GitHub Actions workflow...")

    headers = {
        "Authorization": f"Bearer {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    print(f"🔐 Authorization header set with provided GitHub token.")

    payload = {
        "ref": GITHUB_BRANCH,
        "inputs": {}  # Add custom inputs here if needed
    }
    print(f"📦 Payload prepared to trigger workflow on branch: '{GITHUB_BRANCH}'")

    print(f"🚀 Sending POST request to GitHub API: {GITHUB_API_URL}")
    response = requests.post(GITHUB_API_URL, headers=headers, json=payload)

    if response.status_code == 204:
        print("✅ GitHub Actions workflow triggered successfully!")

        # Update retrain.csv after successful trigger
        try:
            timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')
            df.loc[0, 'Retraining_Decision'] = "NO"
            df.loc[0, 'Rationale'] = f"Retraining Triggered at {timestamp}"
            df.to_csv(RETRAIN_PATH, index=False)
            print(f"✅ Updated retraining decision saved at {RETRAIN_PATH} with decision=NO and rationale including timestamp.")
        except Exception as e:
            print(f"❌ Error updating retraining decision file: {e}")

    else:
        print(f"❌ Failed to trigger GitHub Actions. Status code: {response.status_code}")
        print(f"❌ Response content: {response.text}")
else:
    print("⏭️ No retraining required according to the decision file. No action taken.")
